In [24]:
## libraries required

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas
% matplotlib inline
import time
import cplex
import cvxpy as cp
import cvxopt as copt
# import pygraphviz as pgv
# from nxpd import draw, nxpdParams
# nxpdParams['show'] = 'ipynb'

In [26]:
class MCF_DiGraph:
        
    """
    Extend networkx DiGraph class for our stochastic arc cost purposes
    """
    def __init__(self, residual=False):
        self.nxg = nx.DiGraph()
        self.residual = residual
        self.lam = 0
        self.initial_flow = []
        self.nmcc = []
        self.v_star = None
        self.pi = None
        self.K = None
        
    def set_lambda(self, lam=0):
        self.lam = lam
        self.set_weights()
    
    def get_lambda(self):
        return self.lam
        
    def set_weights(self):
        for u, v, e in self.nxg.edges(data=True):
            self.nxg[u][v]['weight'] = self.nxg[u][v]['mu'] + self.lam*self.nxg[u][v]['var']
    
    def find_feasible(self):
        _,_,soln = get_xMV(0, self)
        i=0
        for u,v,e in self.nxg.edges(data=True):
            self.nxg[u][v]['flow'] = soln[i]
            i += 1
            
    def find_feasible_flow(self):
        """Establish a feasible flow
        """
        # solve max flow problem
        G_MaxFlow = self.nxg.copy()
        G_MaxFlow.add_node('s')
        G_MaxFlow.add_node('t')
        for i, data in G_MaxFlow.nodes(data=True):
            if (i != 's') and (i != 't'):
                b_i = G_MaxFlow.node[i]['demand']
                if b_i > 0:
                    G_MaxFlow.add_edge('s', i, capacity=b_i)
                elif b_i < 0:
                    G_MaxFlow.add_edge(i, 't', capacity=-b_i)
        
        cost, flow = nx.maximum_flow(G_MaxFlow, 's', 't')
        
        # cost, flow = nx.capacity_scaling(self.nxg)
        print("The minimum cost is:", cost)

        # The data structure of flow is not consistent with dictionary data structure 
        # needed for printing the flow vector
        feasible_flow = {}
        for i in self.nxg.nodes(): 
            for j in flow[i].keys():
                if j != 't':
                    feasible_flow[i,j] = flow[i][j]
        
        
        self.initial_flow = feasible_flow
        return feasible_flow
    
    def set_flow_attributes(self, feasible_flow):
        """Set flow variables for the graph
        """
        nx.set_edge_attributes(self.nxg, feasible_flow, 'flow')
            
    def draw_graph(self):
        """Visualize the graph.
        """

        pos = nx.spring_layout(self.nxg)

        # draw nodes, edges and labels
        nx.draw_networkx_nodes(self.nxg, pos, node_size=500, node_color='green', alpha=0.5)
        nx.draw_networkx_edges(self.nxg, pos, width=3, alpha=0.5, edge_color='skyblue')
        nx.draw_networkx_labels(self.nxg, pos, font_size=12, font_family='sans-serif')

        if not self.residual:
            # show the current flow on the arcs
            edge_labels = nx.get_edge_attributes(self.nxg, 'capacity')
        else:
            # show r_ij if it's a residual graph
            edge_labels = nx.get_edge_attributes(self.nxg, 'r')

        nx.draw_networkx_edge_labels(self.nxg, pos, edge_labels=edge_labels)
    
    def build_res_network(self, demand='demand', capacity='capacity', weight='weight', mu='mu', var='var', \
                               flow='flow', jw=False):
        """Build a residual network and initialize a zero flow.
        """
        if sum(self.nxg.nodes[u].get(demand, 0) for u in self.nxg) != 0:
            raise nx.NetworkXUnfeasible("Sum of the demands should be 0.")

        R = MCF_DiGraph(residual=True)
        R.nxg.add_nodes_from((u, {'excess': -self.nxg.nodes[u].get(demand, 0),
                              'potential': 0}) for u in self.nxg)

        inf = float('inf')
        # Detect selfloops with infinite capacities and negative weights.
        for u, v, e in nx.selfloop_edges(self.nxg, data=True):
            if e.get(weight, 0) < 0 and e.get(capacity, inf) == inf:
                raise nx.NetworkXUnbounded(
                    'Negative cost cycle of infinite capacity found. '
                    'Min cost flow may be unbounded below.')

        # Extract edges with positive capacities. Self loops excluded.
        if self.nxg.is_multigraph():
            edge_list = [(u, v, k, e)
                         for u, v, k, e in self.nxg.edges(data=True, keys=True)
                         if u != v and e.get(capacity, inf) > 0]
        else:
    #         edge_list = [(u, v, 0, e) for u, v, e in G.edges(data=True)
    #                      if u != v and e.get(capacity, inf) > 0]
            edge_list = [(u, v, e) for u, v, e in self.nxg.edges(data=True)
                         if u != v and e.get(capacity, inf) > 0]
        # Simulate infinity with the larger of the sum of absolute node imbalances
        # the sum of finite edge capacities or any positive value if both sums are
        # zero. This allows the infinite-capacity edges to be distinguished for
        # unboundedness detection and directly participate in residual capacity
        # calculation.
    #     inf = max(sum(abs(R.nodes[u]['excess']) for u in R),
    #               2 * sum(e[capacity] for u, v, k, e in edge_list
    #                       if capacity in e and e[capacity] != inf)) or 1
        inf = max(sum(abs(R.nxg.nodes[u]['excess']) for u in R.nxg), 2 * sum(e[capacity] for u, v, e in edge_list if capacity in e and e[capacity] != inf)) or 1

        #  for u, v, k, e in edge_list:
        for u, v, e in edge_list:
            r_vu = e.get(flow, 0)
            r_uv = e.get(capacity, inf) - e.get(flow, 0)

            mu_val = e.get(mu, 0)
            var_val = e.get(var, 0)
            flow_val = e.get(flow, 0)
            cap_val = e.get(capacity, 0)
            weight_val = mu_val + self.lam * var_val


            # Add both (u, v) and (v, u) into the residual network marked with the
            # original key. (key[1] == True) indicates the (u, v) is in the
            # original network.


            if not jw:
                if r_uv != 0:
#                     R.add_edge(u, v, key=(k, True), capacity=r_uv, mu=mu, var=var)
                    R.nxg.add_edge(u, v, weight=weight_val, capacity=cap_val, mu=mu_val, var=var_val, r=r_uv)

                if r_vu != 0:
#                     R.add_edge(v, u, key=(k, False), capacity=r_vu, mu=-mu, var=-var)
                    R.nxg.add_edge(v, u, weight=-weight_val, capacity=cap_val, mu=-mu_val, var=-var_val, r=r_vu)
            else:
                if r_uv != 0:
                    R.nxg.add_edge(u, v, r=r_uv, weight=weight_val)
                if r_vu != 0:
                    R.nxg.add_edge(v, u, r=r_vu, weight=-weight_val)

        # Record the value simulating infinity.
        R.nxg.graph['inf'] = inf

        return R

    def nmcc_exists(self, G):
        """Temporary solution
        """
        tol = 1e-10
        valid = []
        for c in nx.simple_cycles(self.nxg):
            if len(c) == 2:
                pass
            else:
                c.append(c[0])
                valid.append(c)
        
        if len(valid) == 0:
            return False
        else:
            
            path_cost = []        
            for c in valid:
                cost = 0
                for i in range(len(c)-1):
                    u = c[i]
                    v = c[i+1]
                    try:
                        cost += self.nxg[u][v]['mu'] + 2*G.lam*G.nxg[u][v]['flow']*self.nxg[u][v]['var']
                    except:
                        cost += self.nxg[u][v]['mu'] + 2*G.lam*G.nxg[v][u]['flow']*self.nxg[u][v]['var']
                path_cost.append(cost)
            
            if  np.min(path_cost) < 0:
                W = valid[np.argmin(path_cost)]
                self.nmcc_cost = np.min(path_cost)
                self.nmcc = W
                
                delta = self.find_delta()
                if delta <= 0:
                    return False
                
#                 print('nmcc_cost: ', self.nmcc_cost)
                if abs(self.nmcc_cost) < tol:
                    self.nmcc_cost = 0.0
                    return False
                else:
                    return True
            else:
                return False
            
    def get_nmcc(self):
        return self.nmcc
        
#     def find_mmc(self):
#         """Temporary solution till MMC is implemented
#         """
#         valid = []
#         for c in nx.simple_cycles(self.nxg):
#             if len(c) == 2:
#                 pass
#             else:
#                 c.append(c[0])
#                 valid.append(c)

#         path_cost = []        
#         for c in valid:
#             cost = 0
#             for i in range(len(c)-1):
#                 u = c[i]
#                 v = c[i+1]
#                 cost += self.nxg[u][v]['weight']
#             path_cost.append(cost)

#         W = valid[np.argmin(path_cost)]

#         self.nmcc = W
#         return W
    
    def find_delta(self):
        min_r = np.inf
        for i in range(len(self.nmcc)-1):
            u = self.nmcc[i]
            v = self.nmcc[i+1]
            rij = self.nxg[u][v]['r']
            min_r = min(min_r, rij)

        return min_r
    
#     def find_uc(self, G):
#         mincneg = np.inf
#         mincpos = np.inf
#         for i in range(len(self.nmcc)-1):
#             u = self.nmcc[i]
#             v = self.nmcc[i+1]
#             try:
#                 dummy = G.nxg[u][v]['flow']
#                 C = 1
#             except:
#                 C = -1
#             if C==1:
#                 mincneg = min(mincneg, G.nxg[u][v]['capacity'] - G.nxg[u][v]['flow'])
#             else:
#                 mincpos = min(mincpos, G.nxg[v][u]['flow'])    
#         return min(mincneg, mincpos)

    def find_xsi_star(self, G):
        nominator = 0
        denominator = 0
        for i in range(len(self.nmcc)-1):
            u = self.nmcc[i]
            v = self.nmcc[i+1]
            
            e = (u, v)
            flow = G.nxg[u][v]['flow'] if G.nxg.has_edge(*e) else G.nxg[v][u]['flow']

            nominator -= self.nxg[u][v]['mu'] + 2*G.lam*flow*self.nxg[u][v]['var']
            denominator += 2*G.lam*abs(self.nxg[u][v]['var'])

        return nominator/denominator

    def remove_no_r(self,u, v):
        if self.nxg[u][v]['r'] == 0:
            self.nxg.remove_edge(u,v)

    def augment_flow(self, min_r):
        for i in range(len(self.nmcc)-1):
            u = self.nmcc[i]
            v = self.nmcc[i+1]
            
            weight = self.nxg[u][v]['weight']
            mu = self.nxg[u][v]['mu']
            var = self.nxg[u][v]['var']
    
            if self.nxg.has_edge(u, v):
                self.nxg[u][v]['r'] -= min_r
                
            self.remove_no_r(u, v)

            if self.nxg.has_edge(v, u):
                self.nxg[v][u]['r'] += min_r
            else:
                self.nxg.add_edge(v, u, r=min_r, weight=-weight, mu=-mu, var=-var)

            self.remove_no_r(v, u)

    def adjust_flow(self, nmcc, delta):

        for i in range(len(nmcc)-1):
            u = nmcc[i]
            v = nmcc[i+1]

            if self.nxg.has_edge(u, v):          
                self.nxg[u][v]['flow'] += delta
            else:
                self.nxg[v][u]['flow'] -= delta
    
    def mu_cost(self):
        cost = 0
        for u,v,e in self.nxg.edges(data=True):
            cost += e.get('flow', 0) * e.get('mu', 0)
            
        return cost

    def var_cost(self):
        
        cost = 0
        for u,v,e in self.nxg.edges(data=True):
            cost += np.square(e.get('flow', 0)) * e.get('var', 0)
            
        return cost
    
    def xi_cost(self):

        cost = 0
        sum_xi = 0
        sum_x = 0
        for u,v,e in self.nxg.edges(data=True):
            cost += e.get('xi', 0) * e.get('var', 0) * e.get('flow', 0)
            sum_xi += e.get('xi', 0)
            sum_x += e.get('flow', 0)
        return cost
    def tot_cost(self):

        cost = 0
        for u,v,e in self.nxg.edges(data=True):
            cost += e.get('flow', 0) * e.get('mu', 0) + self.lam*np.square(e.get('flow', 0)) * e.get('var', 0)

        return cost
    
#     def FindSource(self):
#     ## WIP
#         return 1
    
#     def nmcc_exists(self, G):
#         ## Head
#         g = self.nxg
#         my_inf = np.inf
#         n = len(g.nodes())
#         m = len(g.edges())
#         d_kv = np.ones((n+1, n))
#         d_kv = +my_inf * d_kv
#         pi = np.zeros((n+1, n))
#         Visit = np.ones((n+1, n))
#         M = np.zeros(n)
#         K = np.zeros(n)
#         v_star = None

#         for v in g.nodes():
#             Visit[v-1, 0] = False
#             Visit[v-1, 1] = False

#         s = self.FindSource()
#         d_kv[0, s-1] = 0

#         pi[0, s-1] = None
#         turn = 0
#         Visit[s-1, turn] = True

#         ## Body
#         for k in range(n):
#             for v in g.nodes():
#                 if Visit[v-1, turn] == True:
#                     Visit[v-1, turn] = False
#                     for u in g.neighbors(v):
                        
# #                         try:
# #                             w_vu = g[v][u]['mu'] + 2*G.lam*G.nxg[v][u]['flow']*g[v][u]['var']
# #                         except:
# #                             w_vu = g[v][u]['mu'] + 2*G.lam*G.nxg[u][v]['flow']*g[v][u]['var']
                                
#                         w_vu = g[v][u]['weight']
                        
#                         if d_kv[k+1, u-1] > d_kv[k, v-1] + w_vu:
#                             d_kv[k+1, u-1] = d_kv[k, v-1] + w_vu
#                             pi[k+1, u-1] = v
#                             Visit[u-1, 1-turn] = True
#             turn = 1 - turn

#         ## Tail
#         lam = +my_inf
#         for v in g.nodes():
#             if Visit[v-1, turn] == True:
#                 M[v-1] = -my_inf 
#                 for k in range(n):
#                     if M[v-1] < (d_kv[n, v-1] - d_kv[k, v-1])/(n-k): 
#                         M[v-1] = (d_kv[n, v-1] - d_kv[k, v-1])/(n-k)  
#                         K[v-1] = k

#                 if lam > M[v-1]: 
#                     lam = M[v-1] 
#                     v_star = v
        
#         self.K = K
#         self.v_star = v
#         self.pi = pi
        
#         self.set_nmcc()
                    
#         return True if lam < 0 else False


#     def set_nmcc(self):
          
#         cycle_len = int(len(self.nxg.nodes()) - self.K[self.v_star - 1])
#         self.nmcc = []
#         next_v = self.v_star
        
#         for i in range(cycle_len, 0, -1):
#             self.nmcc.append(int(self.pi[i,next_v-1]))
#             next_v = int(self.pi[i,next_v-1])

#         first = [self.nmcc[-1]]
#         self.nmcc = first + self.nmcc
        


In [27]:
def create_toy_problem(lam):

    # Toy problem generation (With Stochastic Arc Costs) \
    # This creates the graph on pg 318 (Network Flows by Ahuja)

    GAstoch = MCF_DiGraph()
    
    GAstoch.nxg.add_edge(1,2, weight=1,mu=2,var=3,flow=3,capacity=10)
    GAstoch.nxg.add_edge(1,3, weight=10,mu=2,var=3,flow=3,capacity=10)
    GAstoch.nxg.add_edge(4,2, weight=0,mu=2,var=3,flow=3,capacity=10)
    GAstoch.nxg.add_edge(2,3, weight=3,mu=2,var=3,flow=3,capacity=10)
    GAstoch.nxg.add_edge(3,4, weight=2,mu=2,var=3,flow=3,capacity=10)
    GAstoch.nxg.add_edge(4,1, weight=8,mu=2,var=3,flow=3,capacity=10)
    
    GAstoch.nxg.node[1]['demand'] = 1
    GAstoch.nxg.node[2]['demand'] = 0
    GAstoch.nxg.node[3]['demand'] = 0
    GAstoch.nxg.node[4]['demand'] = -1

    GAstoch.nxg.node[1]['pos'] = (0,0)
    GAstoch.nxg.node[2]['pos'] = (3,3)
    GAstoch.nxg.node[3]['pos'] = (3,-3)
    GAstoch.nxg.node[4]['pos'] = (6,0)

    # Lets assign it to another to keep the original
    G = GAstoch
    
    # Set the lambda parameter for mean-variance tradeoff
    G.set_lambda(lam=lam)
    
    # feasible flow x in the network to start with
    initial_flow = G.find_feasible_flow()
    
#     print('feasible_flow: ', initial_flow )

    # set the flow attributes in the graph class
    G.set_flow_attributes(initial_flow)

    return G

def create_random_graph(d_top=10, mu_top=5, var_top=15, num_nodes=5, seed=0):
    """ *** WIP *** 
    Generates random graph with the given network parameters
    """
    ## Build a dataframe with your connections
    # df = pd.DataFrame({ 'from':['A', 'B', 'C','A'], 'to':['D', 'A', 'E','C']})
    # df
    # Build your graph
    # G=nx.from_pandas_dataframe(df, 'from', 'to')

    #Need to make it strongly connected - can always do it adding arcs with very large cost. (MMCC) algo requires!
    #use something like if nx.is_strongly_connected(G5): to check
    #More about generation - satisfy assumptions, guarantee feasibility


    #fix random seed for reproducibility
    np.random.seed(seed)

    # create Directed Graph objec
    G = MCF_DiGraph()

    # number of nodes in the graph
    nodes = np.arange(num_nodes) + 1

    # create an arch between i and i+1, to ensure connectivity and assign max capacity to these arcs to ensure
    # that a feasible flow exists
    for node in nodes:
        if node != len(nodes):
            G.nxg.add_edge(node, node + 1, capacity=d_top, mu=mu_top, var=var_top)
    max_node = len(nodes)

    # print(np.array(list(dict(G.nxg.out_degree(nodes)).values())).mean())
    
    # add additional arcs, and assign capacity, mean, variance - by sampling from a U[0, max_param_value]
    while np.array(list(dict(G.nxg.out_degree(nodes)).values())).mean() < 1.5:
        d = round(np.random.uniform(0,d_top), 2)
        mu = round(np.random.uniform(0, mu_top), 2)
        var = round(np.random.uniform(0, var_top), 2)
        var = round(np.random.normal(var_top, 5, 1)[0],2)
        src = np.random.randint(1, max_node)
        dest = np.random.randint(src+1, max_node+1)
        if not G.nxg.has_edge(src, dest):
            G.nxg.add_edge(src, dest, capacity=d, mu=mu, var=var)

    #print(np.array(list(dict(G.nxg.out_degree(nodes)).values())).mean())
    
    # set supply demand at the nodes
    G.nxg.node[1]['demand'] = d_top
    G.nxg.node[len(nodes)]['demand'] = -d_top
    for i in range(2, len(nodes)):
        G.nxg.node[i]['demand'] = 0
    
    # Set the lambda parameter for mean-variance tradeoff
    G.set_lambda(lam=0.0)
    
    # feasible flow x in the network to start with
#     initial_flow = G.find_feasible_flow()
#     print('feasible_flow: ', initial_flow )

    # set the flow attributes in the graph class
#     G.set_flow_attributes(initial_flow)
    G.find_feasible()
    
    return G


def create_test_instance():
    # Toy problem generation (With Stochastic Arc Costs) \
    # This creates the graph on pg 318 (Network Flows by Ahuja)

    GAstoch = MCF_DiGraph()
    GAstoch.nxg.add_edge(1,2, mu=0, var=0, capacity=10)
    GAstoch.nxg.add_edge(1,3, mu=1, var=1, capacity=10)
    GAstoch.nxg.add_edge(2,3, mu=5, var=5, capacity=10)
    GAstoch.nxg.node[1]['demand'] = 4
    GAstoch.nxg.node[2]['demand'] = 0
    GAstoch.nxg.node[3]['demand'] = -4

    GAstoch.nxg.node[1]['pos'] = (0,0)
    GAstoch.nxg.node[2]['pos'] = (3,3)
    GAstoch.nxg.node[3]['pos'] = (3,-3)

    # Lets assign it to another to keep the original
    G = GAstoch
    
    # Set the lambda parameter for mean-variance tradeoff
    G.set_lambda(lam=1)
    
    # feasible flow x in the network to start with
    initial_flow = G.find_feasible_flow()
    print('feasible_flow: ', initial_flow )

    # set the flow attributes in the graph class
    G.set_flow_attributes(initial_flow)

    return G

In [28]:
def solve_mcf_sa(G, R):

# while R(x) contains a negative cycle do:
#     while nx.negative_edge_cycle(R.nxg):

    while R.nmcc_exists(G):
#         print('There exists a negative marginal cost cycle')
        
        
        nmcc = R.get_nmcc()
#         print('nmcc: ', nmcc)

        # find how much to augment
        
        delta = R.find_delta()
#         print('delta: ', delta)
        
#         delta = R.find_uc(G)
#         print('uc: ', delta)
    
        xsi_star = R.find_xsi_star(G)
#         print('xsi_star: ', xsi_star)

        augment_amount = min(xsi_star, delta)
        
        # augment the flow with the min{r_ij: (i,j) element of the cycle W}
        R.augment_flow(augment_amount)

        # adjust flow (x)
        G.adjust_flow(nmcc, augment_amount)
        



In [29]:
# plt.figure()
# plt.axis('off')
# plt.title('Network')
# G.draw_graph()
# # print(GA.edges(data=True))
# # print("Feasible flow dict", feasible_flow)

# plt.figure()
# plt.axis('off')
# plt.title('Residual Network')
# R.draw_graph()
# print(R.edges(data=True))
f_mid = 100

In [30]:
"""
Bisection search

Given a function f (x) continuous on an interval [a,b] and f (a) * f (b) < 0  
Do  
       c = (a+b)/2  
       if f (a) * f (c) < 0 then  b = c  
                             else  a = c  
"""

lam_bar = 0.4
seed=9
num_nodes = 8

G = create_random_graph(seed=seed, num_nodes=num_nodes)

high = 1.0
low = 0.0
found = False
tol = 1e-20

iters = 0

start = time.time()
while abs(f_mid) > tol:
    
    mid = (high + low)/2.0
    G.set_lambda(lam=mid)
    R = G.build_res_network()
    solve_mcf_sa(G, R)
     
    var_lam = G.var_cost()
    sigma_lam = np.sqrt(var_lam)
    f_mid = mid - lam_bar/(2.0*sigma_lam)
    
    
    G.set_lambda(lam=high)
    R = G.build_res_network()
    solve_mcf_sa(G, R)
     
    var_lam = G.var_cost()
    sigma_lam = np.sqrt(var_lam)
    f_high = high - lam_bar/(2.0*sigma_lam)
    
    if np.sign(f_mid) == np.sign(f_high):
        high = mid
    else:
        low = mid
    iters += 1

    
print('f_lam: ', f_mid)
end = time.time()
time_elapsed = end-start

print('SOLVED')
print('number of iters: ', iters)
print('elapsed time: ', time_elapsed)
print('lambda: ', mid)

lam=mid
G.set_lambda(lam=lam)
R = G.build_res_network()
solve_mcf_sa(G, R)
# flow = nx.get_edge_attributes(G.nxg, 'flow')
print(lam_bar*np.sqrt(G.var_cost()) + G.mu_cost())

f_lam:  0.0
SOLVED
number of iters:  59
elapsed time:  0.6931281089782715
lambda:  0.003015051751597603
177.14564124273656


In [31]:
import pyomo
import pandas
import pyomo.opt
import pyomo.environ as pe
from pyomo.environ import Suffix
from pyomo.core import Var, Constraint
import numpy as np
import logging
from pyomo.environ import *

class MinCostFlow:
    """This class implements a standard min-cost-flow model.  
    It takes as input two csv files, providing data for the nodes and the arcs of the network.  
    The nodes file should have columns: Node, Imbalance
    that specify the node name and the flow imbalance at the node.  The arcs file should have columns:
    Start, End, Cost, UpperBound, LowerBound
    that specify an arc start node, an arc end node, a cost for the arc, and upper and lower bounds for the flow."""
    
    def __init__(self, node_data, arc_data, lam=1):
        """Read in the csv data."""
        self.node_data = node_data
        self.node_data.set_index(['Node'], inplace=True)
        self.node_data.sort_index(inplace=True)
        
        self.arc_data = arc_data
        self.arc_data.set_index(['Start','End'], inplace=True)
        self.arc_data.sort_index(inplace=True)

        self.node_set = self.node_data.index.unique()
        self.arc_set = self.arc_data.index.unique()
        
        neg_dom_data = self.arc_data[self.arc_data['Capacity']==self.arc_data['Flow']]
#         pos_dom_data = self.arc_data[self.arc_data['Flow']==0]
        eq_zer_data = self.arc_data[self.arc_data['Flow']==0]
    
        
        if len(neg_dom_data > 0):
            self.neg_dom_set = neg_dom_data.index.unique()
        else:
            self.neg_dom_set = []
            
#         self.pos_dom_set = pos_dom_data.index.unique()
        self.eq_zer_set = eq_zer_data.index.unique()
#         print(eq_zer_data)
    
        self.lam = lam
        
        self.createModel()
    
    def createModel(self):
        """Create the pyomo model"""
        self.m = pe.ConcreteModel()

        # Create sets
        self.m.neg_dom_set = pe.Set(initialize=self.neg_dom_set, dimen=2)
#         self.m.pos_dom_set = pe.Set(initialize=self.pos_dom_set, dimen=2)
        self.m.eq_zer_set = pe.Set(initialize=self.eq_zer_set, dimen=2)
        self.m.node_set = pe.Set(initialize=self.node_set)
        self.m.arc_set = pe.Set(initialize=self.arc_set , dimen=2)
        
        # Create variables
        self.m.Y = pe.Var(self.m.arc_set, domain=pe.Reals)
        
        def flow_eq_cap(m, n1, n2):
            e = (n1,n2)
#             if self.arc_data.loc[e, 'Flow'] == self.arc_data.loc[e, 'Capacity']:
            return m.Y[e] <= 0
        
#         def flow_eq_zero(m, n1, n2):
#             e = (n1,n2)
# #             if self.arc_data.loc[e, 'Flow'] == 0:
#             return m.Y[e] >= 0

        def flow_noflow(m, n1, n2):
            e = (n1,n2)
            return m.Y[e] == 0
            
        # Lower bounds rule
        def lower_bounds_rule(m, n1, n2):
            e = (n1,n2)
            return m.Y[e] >= self.arc_data.loc[e, 'LowerBound']

        # Upper bounds rule
        def upper_bounds_rule(m, n1, n2):
            e = (n1,n2)
            return m.Y[e] <= self.arc_data.loc[e, 'UpperBound']

        # Flow Balance rule
        def flow_bal_rule(m, n):
            arcs = self.arc_data.reset_index()
            preds = arcs[ arcs.End == n ]['Start']
            succs = arcs[ arcs.Start == n ]['End']
            return sum(m.Y[(p,n)] for p in preds) - sum(m.Y[(n,s)] for s in succs) == self.node_data.loc[n,'Imbalance']

        # Objective rule for the mean variance problem
        def obj_rule(m):
            return sum(self.lam * m.Y[e]**2 * self.arc_data.loc[e, 'Var_Cost'] +
                        m.Y[e] * self.arc_data.loc[e, 'Flow']* self.arc_data.loc[e, 'Var_Cost'] for e in self.arc_set)
        
        self.m.OBJ = pe.Objective(rule=obj_rule, sense=pe.minimize)

        self.m.FlowBal = pe.Constraint(self.m.node_set, rule=flow_bal_rule)
        self.m.FlowCap = pe.Constraint(self.m.neg_dom_set, rule=flow_eq_cap)
#         self.m.FlowZer = pe.Constraint(self.m.pos_dom_set, rule=flow_eq_zero)
        self.m.NoFlow = pe.Constraint(self.m.eq_zer_set, rule=flow_noflow)
        self.m.UpperBound = pe.Constraint(self.m.arc_set, rule=upper_bounds_rule)
        self.m.LowerBound = pe.Constraint(self.m.arc_set, rule=lower_bounds_rule)

    def solve(self):
        """Solve the model."""
        solver = pyomo.opt.SolverFactory("cplex")
        self.m.slack = Suffix(direction=Suffix.IMPORT)
        results = solver.solve(self.m, tee=False, keepfiles=False)

        self.m.solutions.store_to(results)

        
        if (results.solver.status != pyomo.opt.SolverStatus.ok):
            logging.warning('Check solver not ok?')
        if (results.solver.termination_condition != pyomo.opt.TerminationCondition.optimal):  
            logging.warning('Check solver optimality?') 
        return results

In [32]:
def get_xi():
    arc_data = {'Start':[], 'End':[], 'Flow':[], 'Capacity':[], 'Var_Cost':[], 'LowerBound':[], 'UpperBound':[]}
    node_data = {'Node':[], 'Imbalance':[]}
    for u, v, e in G.nxg.edges(data=True):
        arc_data['Start'].append(u)
        arc_data['End'].append(v)
        arc_data['Flow'].append(e.get('flow', 0))
        arc_data['Capacity'].append(e.get('capacity', 0))
        arc_data['Var_Cost'].append(e.get('var', 0))
        arc_data['LowerBound'].append(-e.get('flow',0))
        arc_data['UpperBound'].append(e.get('capacity',0)-e.get('flow',0))

    for u, n in G.nxg.nodes(data=True):

        node_data['Node'].append(u)
        node_data['Imbalance'].append(0)

    arc_df = pandas.DataFrame(data=arc_data)
    node_df = pandas.DataFrame(data=node_data)
    sp = MinCostFlow(node_df, arc_df, lam=lam) 
    results = sp.solve()
    sp.m.solutions.load_from(results)


    for u, v, e in G.nxg.edges(data=True):
        link = (u,v)
        G.nxg[u][v]['xi'] = sp.m.Y[link].value
#         print('edge: %s, flow: %f' % (link, sp.m.Y[link].value))
#         print('edge: %s, xi: %f' % (link, G.nxg[u][v]['xi']))
#         print('edge: %s, flow: %f' % (link, G.nxg[u][v]['flow']))

            
#     for i in range(len(G.nxg.edges)):
#         e = sp.arc_set[i]
#         print('edge: %s, flow: %f' % (e, sp.m.Y[e].value))
        

In [33]:
lam = 0.1
lam_bar = 0.4
seed=9
num_nodes=8
G = create_random_graph(seed=seed, num_nodes=num_nodes)
# G = create_toy_problem(lam=lam)

tol = 1e-20
found=False

iters = 0


start = time.time()

while not found:
    
    G.set_lambda(lam=lam)
    R = G.build_res_network()
    solve_mcf_sa(G, R)
    
    var_cost = G.var_cost()
    
    f_lam = lam - lam_bar/(2*sqrt(var_cost))
    
    get_xi()
    var_cost_der = G.xi_cost()
    
    f_lam_der = 1 + (lam_bar/2)*(var_cost**(-3/2))*var_cost_der

    lam_prev = lam
    lam = lam_prev - f_lam/f_lam_der
    
    diff = lam - lam_prev
    iters += 1
    if abs(diff) < tol:
        found = True
        
end = time.time()
time_elapsed = end - start    
print('SOLVED')
print('num iters: ', iters)
print('time elapsed: ', time_elapsed)
print('lambda: ', lam)

G.set_lambda(lam=lam)
R = G.build_res_network()
solve_mcf_sa(G, R)
flow = nx.get_edge_attributes(G.nxg, 'flow')
print(lam_bar*np.sqrt(G.var_cost()) + G.mu_cost())


TypeError: Cannot infer number of levels from empty list

In [34]:
import pyomo
import pandas
import pyomo.opt
import pyomo.environ as pe
from pyomo.environ import Suffix
from pyomo.core import Var, Constraint
import numpy as np
import logging
from pyomo.environ import *
# from pyomo.util.infeasible import log_infeasible_constraints

class MinCostFlowMV:
    """This class implements a standard min-cost-flow model.  
    It takes as input two csv files, providing data for the nodes and the arcs of the network.  
    The nodes file should have columns: Node, Imbalance
    that specify the node name and the flow imbalance at the node.  The arcs file should have columns:
    Start, End, Cost, UpperBound, LowerBound
    that specify an arc start node, an arc end node, a cost for the arc, and upper and lower bounds for the flow."""
    
    def __init__(self, node_data, arc_data, lam=1):
        """Read in the csv data."""
        self.node_data = node_data
        self.node_data.set_index(['Node'], inplace=True)
        self.node_data.sort_index(inplace=True)
        
        self.arc_data = arc_data
        self.arc_data.set_index(['Start','End'], inplace=True)
        self.arc_data.sort_index(inplace=True)
        self.arc_set = self.arc_data.index.unique()

        self.node_set = self.node_data.index.unique()
        

        self.lam = lam
        self.createModel()
    
    def createModel(self):
        """Create the pyomo model"""
        self.m = pe.ConcreteModel()

        # Create sets
        self.m.node_set = pe.Set(initialize=self.node_set)
        self.m.arc_set = pe.Set(initialize=self.arc_set , dimen=2)

        # Create variables
        self.m.Y = pe.Var(self.m.arc_set, domain=pe.Reals)
        self.m.GMO = pe.Var(self.m.arc_set, domain=pe.Reals)
            
        # Lower bounds rule
        def lower_bounds_rule(m, n1, n2):
            e = (n1,n2)
            return m.Y[e] >= self.arc_data.loc[e, 'LowerBound']

        # Upper bounds rule
        def upper_bounds_rule(m, n1, n2):
            e = (n1,n2)
            return m.Y[e] <= self.arc_data.loc[e, 'UpperBound']
    
#         def new_var_rule(m, n1, n2):
#             e = (n1,n2)
#             return m.GMO[e] == self.arc_data.loc[e, 'Var_Cost'] * m.Y[e]
        # Flow Balance rule
        
        def flow_bal_rule(m, n):
            arcs = self.arc_data.reset_index()
            preds = arcs[ arcs.End == n ]['Start']
            succs = arcs[ arcs.Start == n ]['End']
            return sum(m.Y[(p,n)] for p in preds) - sum(m.Y[(n,s)] for s in succs) ==  - self.node_data.loc[n,'Imbalance']

        # Objective rule for the mean variance problem
        def obj_rule(m):
            return sum(m.Y[e]*self.arc_data.loc[e, 'Mean_Cost']  + 
                       self.lam*(m.Y[e]**2) * self.arc_data.loc[e, 'Var_Cost']  for e in self.arc_set)
        
        self.m.OBJ = pe.Objective(rule=obj_rule, sense=pe.minimize)

        self.m.FlowBal = pe.Constraint(self.m.node_set, rule=flow_bal_rule)
        self.m.UpperBound = pe.Constraint(self.m.arc_set, rule=upper_bounds_rule)
        self.m.LowerBound = pe.Constraint(self.m.arc_set, rule=lower_bounds_rule)
        
    def solve(self):
        """Solve the model."""
        solver = pyomo.opt.SolverFactory("cplex")
        self.m.slack = Suffix(direction=Suffix.IMPORT)
        results = solver.solve(self.m, tee=False, keepfiles=False, symbolic_solver_labels=True)

        self.m.solutions.store_to(results)
#         log_infeasible_constraints(self.m)
        
        if (results.solver.status != pyomo.opt.SolverStatus.ok):
            logging.warning('Check solver not ok?')
        if (results.solver.termination_condition != pyomo.opt.TerminationCondition.optimal):  
            logging.warning('Check solver optimality?') 
        return results

In [35]:
def get_xMV(lam, G):
    arc_data = {'Start':[], 'End':[], 'Mean_Cost':[], 'Var_Cost':[], 'LowerBound':[], 'UpperBound':[]}
    node_data = {'Node':[], 'Imbalance':[]}
    for u, v, e in G.nxg.edges(data=True):
        arc_data['Start'].append(u)
        arc_data['End'].append(v)
        arc_data['Mean_Cost'].append(e.get('mu', 0))
        arc_data['Var_Cost'].append(e.get('var',0))
        arc_data['LowerBound'].append(0)
        arc_data['UpperBound'].append(e.get('capacity',0))

    
    for u, n in G.nxg.nodes(data=True):
        node_data['Node'].append(u)
        node_data['Imbalance'].append(n.get('demand',0))
 

    arc_df = pandas.DataFrame(data=arc_data)
    node_df = pandas.DataFrame(data=node_data)
    sp = MinCostFlowMV(node_df, arc_df, lam=lam) 
    results = sp.solve()
    sp.m.solutions.load_from(results)

    var_cost = 0
    mean_cost = 0
    soln = []
    for u, v, e in G.nxg.edges(data=True):
        link = (u,v)
        var_cost += sp.m.Y[link].value**2 * e.get('var',0)
        mean_cost += sp.m.Y[link].value * e.get('mu',0)
        soln.append(sp.m.Y[link].value)
    return var_cost, mean_cost, soln

In [1]:
"""
Bisection search
"""

seed=9
lam_bar = 0.4
num_nodes = 8

G = create_random_graph(seed=seed, num_nodes=num_nodes)
# G = create_toy_problem(lam=lam)

high = 1
low = 0
found = False
tol = 1e-30

iters = 0

start = time.time()
while f_mid > tol:
    
    
    G.set_lambda(lam=high)
    var_lam, _, _ = get_xMV(high, G)
     
    sigma_lam = np.sqrt(var_lam)
    f_high = high - float(lam_bar)/float((2*sigma_lam))

    mid = (high+low)/2.0
    G.set_lambda(lam=mid)
    var_lam, _, _ = get_xMV(mid, G)
     
    sigma_lam = np.sqrt(var_lam)
    f_mid = mid - float(lam_bar)/float((2*sigma_lam))
    
    if np.sign(f_mid) == np.sign(f_high):
        high = mid
    else:
        low = mid
    iters += 1

end = time.time()
time_elapsed = end-start

print('SOLVED')
print('number of iters: ', iters)
print('elapsed time: ', time_elapsed)
print('lambda: ', lam)

G.set_lambda(lam=mid)
var_lam = get_xMV(mid, G)
varcost, meancost, soln = get_xMV(mid, G)
print(lam_bar*np.sqrt(varcost)+meancost)
print('====')
print('soln: ')
print(soln)

NameError: name 'create_random_graph' is not defined

In [5]:
# Problem data.
import numpy as np
import cvxpy as cp

seed=9
lam_bar = 0.4
num_nodes = 8
G = create_random_graph(seed=seed, num_nodes=num_nodes)
G.set_lambda(lam=lam_bar)

m = G.nxg.number_of_edges()
n = G.nxg.number_of_nodes()
        
mu = np.zeros(m)
sigma = np.zeros(m)
cap = np.zeros(m)
d = np.zeros(n)
F = np.zeros((m,n))
x = cp.Variable(m)
x_n = cp.Variable(m)

i = 0
arc_dict = {}
for u, v, e in G.nxg.edges(data=True):
    mu[i] = e.get('mu',0)
    sigma[i] = np.sqrt(e.get('var',0))
    cap[i] = e.get('capacity',0)
    arc_dict[i] = (u, v)
    i += 1

print(arc_dict)
i = 0
for node, dat in G.nxg.nodes(data=True):
    d[i] = dat['demand']
    i += 1

    arc_data = {'Start':[], 'End':[]}
for u, v, e in G.nxg.edges(data=True):
    arc_data['Start'].append(u)
    arc_data['End'].append(v)
    
constraints = [0 <= x, x <= cap]
for k in range(m):
    constraints.append(x_n[k] == sigma[k]*x[k])
        
arc_data = pandas.DataFrame(data=arc_data)

arc_data.set_index(['Start','End'], inplace=True)
arc_data.sort_index(inplace=True)
# self.arc_set = self.arc_data.index.unique()      

arcs = arc_data.reset_index()


for i in range(n):
    preds = []
    succs = []
    for key, vals in arc_dict.items():
        if vals[0] == i+1:
            preds.append(key)
        if vals[1] == i+1:
            succs.append(key)
    print(i)
    print("=====")
    print(preds)
    print(succs)
    
    constraint = sum(x[p] for p in preds) - sum(x[s] for s in succs) ==  d[i]
    print(constraint)
    constraints.append(constraint)
    


# Construct the problem.

exp_val = mu.T*x
objective = cp.Minimize(exp_val + lam_bar*cp.norm(x_n, 2))


prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve(solver=cp.MOSEK, verbose=True)
print("status:", prob.status)
# The optimal value for x is stored in `x.value`.
print(x.value)
print(objective.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
# print(constraints[0].dual_value)

NameError: name 'create_random_graph' is not defined

In [4]:
import cvxpy as cp
print (cp.installed_solvers())

['SCS', 'MOSEK', 'ECOS_BB', 'ECOS', 'CVXOPT', 'OSQP', 'CPLEX']
